In [1]:
import os
import sys
import numpy as np
import pandas as pd

sys.path.append('/mnt/data1/spatial/tissue-alignment/traversal')
from io_utils import read_image, read_spatial_data

%matplotlib inline

dz = 'UC'
section = 'B'
section_path = f'/mnt/data1/spatial/data/colon/{dz}/{section}'
belayer_path = f'colon/colon-{dz}-{section}-belayer.csv'
out_count_csv = f'belayer_{dz}_{section}_counts.csv'
out_depth_csv = f'belayer_{dz}_{section}_depths.csv'
out_norm_count_csv = f'belayer_{dz}_{section}_normalized_path_counts.csv'

In [2]:
fullres = False
im = read_image(section_path, fullres)
pos_df, spot_radius = read_spatial_data(section_path, fullres)

In [3]:
belayer = pd.read_csv(belayer_path)
pos_df['depth2'] = belayer['depth2']

In [4]:
from graph import compute_path_counts
from io_utils import read_transcription_data

counts = read_transcription_data(section_path)

dummy_path_idxs = [-1] * (pos_df['depth2'].max() + 1) # assumes there are no gaps in depth2, see prior assert
mean_counts, std_counts = compute_path_counts(
    pos_df=pos_df,
    counts=counts,
    path_idxs=dummy_path_idxs,
    clusters=pos_df['depth2'],
)

QUEUEING TASKS | :   0%|          | 0/173 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/173 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/173 [00:00<?, ?it/s]

In [5]:
mean_counts.to_csv(out_count_csv, index=False)

In [6]:
normalized_length = 101

xp = np.linspace(0, 1, len(mean_counts))
_mean_counts = mean_counts.dropna(axis=0)
xp = xp[_mean_counts.index.to_numpy()]
x = np.linspace(0, 1, normalized_length)
out = []
for i in range(_mean_counts.shape[1]):
    interp = np.interp(x, xp, _mean_counts.iloc[:, i].to_numpy())
    out.append(interp)
path_normalized_counts = pd.DataFrame(np.asarray(out).T, columns=mean_counts.columns)
path_normalized_counts.to_csv(out_norm_count_csv, index=False)